In [1]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

boston = load_boston()
dfX = pd.DataFrame(boston.data, columns=boston.feature_names)
dfy = pd.DataFrame(boston.target, columns=["MEDV"])
df = pd.concat([dfX, dfy], axis=1)

#학습용과 검증용을 7:3으로 구분
N = len(df)
ratio = 0.7
np.random.seed(0)
idx_train = np.random.choice(np.arange(N), np.int64(ratio * N), replace=False)
idx_test = list(set(np.arange(N)).difference(idx_train))

df_train = df.iloc[idx_train]
df_test = df.iloc[idx_test]

c:\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.
    :fu

In [2]:
import statsmodels.api as sm

model = sm.OLS.from_formula("MEDV ~ " + "+".join(boston.feature_names), data=df_train)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       0.728
Model:                            OLS   Adj. R-squared:                  0.718
Method:                 Least Squares   F-statistic:                     70.06
Date:                Thu, 09 Mar 2023   Prob (F-statistic):           8.57e-88
Time:                        10:55:02   Log-Likelihood:                -1043.0
No. Observations:                 354   AIC:                             2114.
Df Residuals:                     340   BIC:                             2168.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     35.0719      5.932      5.913      0.000      23.404      46.739
CRIM          -0.1005      0.035     -2.869      0.004      -0.169      -0.032
ZN             0.0381      0.017      2.248      0.025       0.005       0.071
INDUS          0.0202      0.072      0.281      0.779      -0.121       0.162
CHAS           1.1498      1.020      1.127      0.260      -0.856       3.156
NOX          -17.3942      4.522     -3.847      0.000     -26.288      -8.500
RM             3.8640      0.485      7.959      0.000       2.909       4.819
AGE            0.0004      0.016      0.023      0.982      -0.030       0.031
DIS           -1.3285      0.236     -5.626      0.000      -1.793      -0.864
RAD            0.3741      0.084      4.447      0.000       0.209       0.540
TAX           -0.0160      0.005     -3.315      0.001      -0.025      -0.007
PTRATIO       -0.8989      0.153     -5.885      0.000      -1.199      -0.598
B              0.0095      0.003      3.015      0.003       0.003       0.016
LSTAT         -0.5013      0.060     -8.423      0.000      -0.618      -0.384
==============================================================================
Omnibus:                      136.641   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              602.833
Skew:                           1.618   Prob(JB):                    1.25e-131
Kurtosis:                       8.513   Cond. No.                     1.47e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.47e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [3]:
#검증용 데이터셋으로 모형 평가
pred = result.predict(df_test)

rss = ((df_test.MEDV - pred) ** 2).sum()
tss = ((df_test.MEDV - df_test.MEDV.mean()) ** 2).sum()
rsquared = 1 - rss/tss
rsquared

0.7519796502601109

In [4]:
from sklearn.model_selection import train_test_split

#학습용, 검증용 구분
df_train, df_test = train_test_split(df, test_size=0.3, random_state=0)
df_train.shape, df_test.shape

((354, 14), (152, 14))

In [5]:
#학습용X,y, 검증용X,y 구분
dfX_train, dfX_test, dfy_train, dfy_test = train_test_split(
    dfX, dfy, test_size=0.3, random_state=0)
dfX_train.shape, dfy_train.shape, dfX_test.shape, dfy_test.shape

((354, 13), (354, 1), (152, 13), (152, 1))

In [7]:
from sklearn.model_selection import KFold

scores = np.zeros(5)
cv = KFold(5, shuffle=True, random_state=0)
for i, (idx_train, idx_test) in enumerate(cv.split(df)):
    df_train = df.iloc[idx_train]
    df_test = df.iloc[idx_test]

    model = sm.OLS.from_formula("MEDV ~ " + "+".join(boston.feature_names), data=df_train)
    result = model.fit()

    pred = result.predict(df_test)
    rss = ((df_test.MEDV - pred) ** 2).sum()
    tss = ((df_test.MEDV - df_test.MEDV.mean()) ** 2).sum()
    rsquared = 1 - rss/tss

    scores[i] = rsquared
    print(f"학습용 R2 = {result.rsquared:.3f}, 검증용 R2 = {rsquared:.3f}")

학습용 R2 = 0.773, 검증용 R2 = 0.589
학습용 R2 = 0.729, 검증용 R2 = 0.778
학습용 R2 = 0.749, 검증용 R2 = 0.668
학습용 R2 = 0.757, 검증용 R2 = 0.668
학습용 R2 = 0.705, 검증용 R2 = 0.840


In [11]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

scores1 = np.zeros(5)
scores2 = np.zeros(5)
scores3 = np.zeros(5)
cv = KFold(5, shuffle=True, random_state=0)

for i, (idx_train, idx_test) in enumerate(cv.split(df)):
    df_train = df.iloc[idx_train]
    df_test = df.iloc[idx_test]

    model = sm.OLS.from_formula("MEDV ~ " + "+".join(boston.feature_names), data=df_train)
    result = model.fit()

    pred = result.predict(df_test)
    #결정계수
    rsquared = r2_score(df_test.MEDV, pred)
    scores1[i] = rsquared
    
    #평균제곱오차
    mse = mean_squared_error(df_test.MEDV, pred)
    scores2[i] = mse

    #평균절대오차
    mae = mean_absolute_error(df_test.MEDV, pred)
    scores3[i] = mae

print(scores1) #rsquare
print(scores2) #mse
print(scores3) #mae
print(np.mean(scores1))
print(np.mean(scores2))
print(np.mean(scores3))

[0.58922238 0.77799144 0.66791979 0.6680163  0.83953317]
[33.44898    18.65881615 21.23463289 29.22251557 16.57369039]
[3.84290922 3.38979394 3.07473854 3.6463452  3.03058651]
0.7085366175182802
23.82772699990077
3.396874681157459


In [14]:
from sklearn.base import BaseEstimator, RegressorMixin
import statsmodels.formula.api as smf
import statsmodels.api as sm

class StatsmodelsOLS(BaseEstimator, RegressorMixin):
    def __init__(self, formula):
        self.formula = formula
        self.model = None
        self.data = None
        self.result = None

    def fit(self, dfX, dfy):
        self.data = pd.concat([dfX, dfy], axis=1)
        self.model = smf.ols(self.formula, data=self.data)
        self.result = self.model.fit()

    def predict(self, new_data):
        return self.result.predict(new_data)

In [17]:
from sklearn.model_selection import cross_val_score

model = StatsmodelsOLS("MEDV ~ " + "+".join(boston.feature_names))
cv = KFold(5, shuffle=True, random_state=0)
cross_val_score(model, dfX, dfy, scoring="r2", cv=cv)

array([0.58922238, 0.77799144, 0.66791979, 0.6680163 , 0.83953317])

In [21]:
#평균제곱오차로 평가하는 경우
result = cross_val_score(model, dfX, dfy, scoring='neg_mean_squared_error', cv=cv)

#음수로 나온 결과값을 양수로 변환
rmse_score = np.sqrt(-result)
rmse_score

array([5.78350932, 4.31958518, 4.60810513, 5.40578538, 4.07107976])